In [1]:
import pandas as pd
from datetime import datetime
from glob import glob 
files = glob("/dccstor/srosent2/generative/appen/round2_jobs*/output/*json.zip")
#files = glob("/dccstor/srosent2/generative/appen/round1_jobs/*json.zip") #["job_1999101.json","job_1988758.json", "job_2004889.json","job_2006984.json", "job_2022794.json", "job_2035917.json"]

seen_ids = set()

for file_name in files:
    # file_name = "/dccstor/srosent2/generative/appen/round1_jobs/" + file
    data = pd.read_json(file_name, lines=True)

    for index, row in data.iterrows():
        info = row['results']['judgments'][0]
        # print(row['results'])
            
        seen_ids.add(info['unit_data']['question_id'])

print(len(seen_ids))
# print(seen_ids)

KeyError: 'question_id'

In [6]:
import gzip
import json

def load_json_from_file(gt_file_patterns):
    data = []
    if gt_file_patterns.endswith('gz'):
        f = gzip.open(gt_file_patterns, 'rt', encoding='utf-8')
    else:
        f = open(gt_file_patterns, 'rt', encoding='utf-8')
    lines = f.readlines()
    for line in lines:
        data.append(json.loads(line))
    return data

In [4]:
# load tydi json
import glob
import spacy

# spacy.cli.download('en_core_web_sm')

nlp = spacy.load('en_core_web_sm')

train_data_dir = "/dccstor/srosent2/primeqa/data/nq-lfqa-train/*jsonl"
dev_data_dir = "/dccstor/srosent2/primeqa/data/dev/nq-lfqa/*jsonl"

train_files = glob.glob(train_data_dir)
dev_files = glob.glob(dev_data_dir)



In [28]:
train_data = []
for train_file in train_files:
    train_data.extend(load_json_from_file(train_file))

In [7]:
dev_data = []
for dev_file in dev_files:
    dev_data.extend(load_json_from_file(dev_file))

print(len(dev_data))

seen = 0
not_la = 0
seen_dev_ids = set()
for example in dev_data:
    
    qtype = example['type'][0]
    
    if qtype != 'la':
        not_la += 1
        continue
    if str(example['example_id']) in seen_ids:
        seen_dev_ids.add(example['example_id'])
        seen += 1
print(not_la)
print(seen)
print(len(seen_dev_ids))

3393
2265
0
0


In [15]:
train_files

['/dccstor/srosent2/primeqa/data/nq-lfqa-train/nq-train-tydiformat4.jsonl',
 '/dccstor/srosent2/primeqa/data/nq-lfqa-train/nq-train-tydiformat2.jsonl',
 '/dccstor/srosent2/primeqa/data/nq-lfqa-train/nq-train-tydiformat0.jsonl',
 '/dccstor/srosent2/primeqa/data/nq-lfqa-train/nq-train-tydiformat1.jsonl',
 '/dccstor/srosent2/primeqa/data/nq-lfqa-train/nq-train-tydiformat.jsonl',
 '/dccstor/srosent2/primeqa/data/nq-lfqa-train/nq-train-tydiformat3.jsonl']

In [12]:
len(train_data)

61499

In [4]:
import csv

# part1 == also 2000?
# up to 19505 to get part2 (2000 each)
# up to 30209 to get part3 (2000 each)
# up to 41213 to get part4 (2000 each)
# up to 53351 to get part5 (2000 each)
# 64067 part6 (2000)
# to end part7 64504 (175)

def is_majority_type(types, type):

    count = 0

    for t in types:
        if t == type:
            count += 1
    if count >= 3:
        return True
    return False

def get_appen_data(data, type, exclude_ids, output_file, start):
    f = open(output_file, 'w', encoding='UTF8', newline='')
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(["question_id","question","title","url","long_answer","minimal_text"])

    count = 0
    index = start
    lines = []
    excluded = 0
    for example in data[start:]:
        index += 1
        # if index >= 19505:
        #     continue
        # qtype = example['type'][0]
        
        # if qtype != type:
        #     continue

        if not is_majority_type(example['type'], type):
            continue
        
        example_id = example['example_id']

        if str(example_id) in exclude_ids:
            excluded += 1
            continue

        question = example["question_text"]

        q_word = question.split()[0]

        annotation = example['annotations'][0]

        if annotation == None:
            continue

        minimal_text = ""
        if annotation['minimal_answer']['plaintext_start_byte'] != -1:
            minimal_text = example['document_plaintext'].encode('utf-8')[annotation['minimal_answer']['plaintext_start_byte']:annotation['minimal_answer']['plaintext_end_byte']].decode('utf-8')
            continue
        passage_offsets = example['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
        passage_text = example['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
        
        passage_sentences = ""
        sentences = nlp(passage_text)
        
        num_sentences = 0
        for sentence in sentences.sents:
            num_sentences += 1
            passage_sentences += sentence.text + "&nbsp;"

        if num_sentences < 5:
            continue

        count += 1
        if count % 100 == 0:
            print(count)
            print(index)
        if count > 2000:
            break

        writer.writerow([str(example_id), question, example['document_title'], example['document_url'], passage_sentences, minimal_text])
    print(count)
    print(excluded)
    f.close()
           

In [5]:
get_appen_data(train_data, "la", seen_ids, "/dccstor/srosent2/generative/data_for_appen/full_nq_task_nomin_train_more.tsv", 0)
# get_appen_data(dev_data, "la", seen_ids, "/dccstor/srosent2/generative/data_for_appen/full_nq_task_nomin_dev.tsv", 0)

NameError: name 'train_data' is not defined

In [10]:
# load final data and split into train and dev
import pandas as pd

annotated_data = pd.read_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_multiple-9.15.23.json", dtype={'id':str}, lines=True)
annotated_data.sample(5)

,id,input,passages,output
1365,-8723799487992974106,where is the new king arthur movie filmed,"[{'title': 'King Arthur: Legend of the Sword',...",[{'answer': 'Filming for King Arthur: Legend o...
1242,-8602381519038886159,what happens at the end of where the wild thin...,"[{'title': 'Where the Wild Things Are (film)',...",[{'answer': 'Max is going home because he is n...
2034,-5060252514048143431,how do you write i am in hebrew,"[{'title': 'I Am that I Am', 'text': 'I Am tha...",[{'answer': 'I Am that I Am ( אֶהְיֶה אֲשֶׁר א...
1491,7253045048184834906,average height and weight of kentucky derby jo...,"[{'title': 'Jockey', 'text': 'Jockeys must be ...",[{'answer': 'The Kentucky Derby has a weight l...
659,-1152407807275770327,show me on a map where ecuador is,"[{'title': 'Ecuador', 'text': 'Ecuador ( / ˈɛk...",[{'answer': 'Ecuador is a representative democ...


In [8]:
dev_ids = []

for item in dev_data:
    dev_ids.append(str(item['example_id']))

print(len(dev_ids))
print(dev_ids[0:5])

3393
['-6570496346595660652', '4111902318448915849', '3599421094587589904', '3367999921663366204', '-2103986527722712835']


In [11]:
dev_annotated_data = annotated_data[annotated_data['id'].isin(dev_ids)]
train_annotated_data = annotated_data[~annotated_data['id'].isin(dev_ids)]

train_annotated_data.to_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_multiple-9.15.23_train.json", lines=True, orient='records')
dev_annotated_data.to_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_multiple-9.15.23_dev.json", lines=True, orient='records')

print(len(annotated_data))
print(len(dev_annotated_data))
print(len(train_annotated_data))

NameError: name 'annotated_data' is not defined

In [16]:
single_annotated_data = annotated_data[annotated_data['id'].isin(dev_ids)]

print(len(single_annotated_data))

303


In [23]:
annotated_data_first_word  = annotated_data['input'].str.split().str.get(0)
annotated_data_first_word.value_counts()[0:30]


what          642
where         498
who           384
when          243
how           174
why            98
the            46
which          19
what's         17
is             15
difference     13
in             10
describe        9
meaning         8
summary         8
can             7
do              6
a               5
explain         4
does            4
give            3
last            3
list            3
whats           3
song            3
indian          3
average         2
role            2
state           2
rhyme           2
Name: input, dtype: int64

In [24]:
# load final data and split into train and dev
import pandas as pd

annotated_single_data = pd.read_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_single-9.11.23.json", dtype={'id':str}, lines=True)

annotated_single_data_first_word  = annotated_single_data['input'].str.split().str.get(0)
annotated_single_data_first_word.value_counts()[0:30]

what          2827
who           2414
where         2059
when          1464
how            904
why            455
the            256
which          143
is              91
in              77
what's          54
a               49
difference      49
meaning         33
summary         26
can             26
does            23
do              23
describe        22
explain         20
whats           19
who's           19
name            19
list            16
during          11
are             10
an              10
main             9
did              9
this             9
Name: input, dtype: int64

In [42]:
sa_train_data = {}

def is_majority_type(types, type):

    count = 0

    for t in types:
        if t == type:
            count += 1
    if count >= 3:
        return True
    return False

print(len(train_data))
count_sa = 0

for d in train_data:
    first_word = d['question_text'].split()[0]
    if d['type'][0] == 'sa':
        if first_word not in sa_train_data:
            sa_train_data[first_word] = 1
        else:
            sa_train_data[first_word] += 1
        count_sa += 1
    

print(count_sa)
print("SA")
for word in sa_train_data:
    if sa_train_data[word] > 3:
        print(f'{word} {sa_train_data[word]}')

61499
0
SA


In [4]:
import glob
train_files = glob.glob("/dccstor/srosent2/primeqa/data/train/nq-full/*.jsonl.gz")

train_data_full = []
for train_file in train_files:
    train_data_full.extend(load_json_from_file(train_file))

train_data_full[0]

{'document_plaintext': "Over the Rainbow - wikipedia Over the Rainbow This article is about the 1939 song . For other uses , see Over the Rainbow ( disambiguation ) . `` Over the Rainbow '' Song by Judy Garland Published 1939 Genre Ballad Composer ( s ) Harold Arlen Lyricist ( s ) E.Y. Harburg `` Over the Rainbow '' is a ballad , with music by Harold Arlen and lyrics by Yip Harburg . It was written for the movie The Wizard of Oz and was sung by actress Judy Garland , in her starring role as Dorothy Gale . It won the Academy Award for Best Original Song and became Garland 's signature song , as well as one of the most enduring standards of the 20th century . About five minutes into the film , Dorothy sings the song after failing to get Aunt Em , Uncle Henry , and the farmhands to listen to her relate an unpleasant incident involving her dog , Toto , and the town spinster , Miss Gulch ( Margaret Hamilton ) . Aunt Em tells her to `` find yourself a place where you wo n't get into any trou

In [10]:
len(list(train_data_full))

307373

In [4]:
def get_type(annotation):
    if annotation['passage_answer']['candidate_index'] == -1:
        return 'na'
    if annotation['minimal_answer']['plaintext_start_byte'] > -1:
        return 'sa'
    if annotation['yes_no_answer'] != 'NONE':
        return 'bool'
    else:
        return 'la'
    

sa_train_data = {}
count_sa = 0
na_train_data = {}
count_na = 0
example_ids = set()
counts = {'la':0,'bool':0,'sa':0,'na':0}

for d in train_data_full:
    if d['example_id'] in example_ids:
        print(f"duplicate {d['example_id']}")
        continue
    example_ids.add(d['example_id'])
    first_word = d['question_text'].split()[0]
    answer_type = get_type(d['annotations'][0])
    counts[answer_type] += 1
    if answer_type == 'sa':
        if first_word not in sa_train_data:
            sa_train_data[first_word] = 1
        else:
            sa_train_data[first_word] += 1
        count_sa += 1
    elif answer_type == 'na':
        if first_word not in na_train_data:
            na_train_data[first_word] = 1
        else:
            na_train_data[first_word] += 1
        count_na += 1
    

print(count_sa)
print("SA")
for word in sa_train_data:
    if sa_train_data[word] > 30:
        print(f'{word} {sa_train_data[word]}')
print(count_na)
print("NA")
for word in na_train_data:
    if na_train_data[word] > 30:
        print(f'{word} {na_train_data[word]}')
print(count_na)
print(len(example_ids))
print(counts)

106926
SA
who 38047
where 12144
how 5680
when 20433
what 15195
name 304
what's 401
why 526
which 1938
the 2534
first 163
in 653
this 78
an 73
if 43
at 79
number 99
who's 338
all 34
when's 84
a 358
total 71
top 53
is 203
where's 33
one 69
according 68
whats 91
game 35
most 199
during 64
last 91
meaning 50
actress 54
i 45
on 48
highest 58
list 98
real 41
actor 67
song 33
movie 35
155225
NA
where 13048
what 22132
the 7430
1 40
who 29623
when 14952
office 33
how 9341
this 227
which 6362
can 415
in 1364
are 223
national 108
little 49
oh 48
game 99
meaning 185
discuss 230
download 81
explain 583
write 316
name 537
tell 35
who's 327
describe 552
as 136
all 412
an 214
most 235
song 263
now 95
john 75
history 146
is 1109
summary 194
lyrics 148
youtube 47
show 206
you 241
what's 455
why 2265
names 78
if 334
black 68
it's 118
different 83
list 1228
that's 42
first 157
using 36
map 147
a 1218
does 316
was 93
american 93
blue 33
to 97
i 933
on 118
one 245
love 77
watch 61
michael 62
queen 34
india 

In [1]:
import spacy 

# spacy.cli.download('en_core_web_sm')
nlp = spacy.load('en_core_web_sm')

In [3]:
import glob
train_files = glob.glob("/dccstor/srosent2/primeqa/data/nq-lfqa-train/*.jsonl")
train_data_long = []
for train_file in train_files:
    train_data_long.extend(load_json_from_file(train_file))

train_data_long[0]

{'document_plaintext': "Criminal justice - wikipedia Criminal justice Jump to : navigation , search For the 1990 film , see Criminal Justice ( film ) . For the 2008 television series , see Criminal Justice ( TV series ) . The examples and perspective in this article may not represent a worldwide view of the subject . You may improve this article , discuss the issue on the talk page , or create a new article , as appropriate . ( December 2010 ) ( Learn how and when to remove this template message ) Criminology and penology Theory ( show ) Anomie Biosocial criminology Broken windows Collective efficacy Crime analysis Criminalization Differential association Deviance Labeling theory Psychopathy Rational choice Social control Social disorganization Social learning Strain Subculture Symbolic interactionism Victimology Types of crime ( show ) Against humanity Blue - collar Corporate Juvenile Organized Political Public - order State State - corporate Victimless White - collar War Penology ( h

In [5]:

import glob
dev_files = glob.glob("/dccstor/srosent2/primeqa/data/dev/nq-lfqa/*.jsonl")
dev_data_long = []
for dev_file in dev_files:
    dev_data_long.extend(load_json_from_file(dev_file))

dev_data_long[0]

{'document_plaintext': "Agriculture in the United States - wikipedia Agriculture in the United States Jump to : navigation , search A wheat harvest in Idaho This photo from a 1921 encyclopedia shows a tractor plowing a crop field . Agriculture is a major industry in the United States , which is a net exporter of food . As of the 2007 census of agriculture , there were 2.2 million farms , covering an area of 922 million acres ( 3,730,000 km ) , an average of 418 acres ( 169 hectares ) per farm . Although agricultural activity occurs in all states , it is particularly concentrated in the Great Plains , a vast expanse of flat , arable land in the center of the United States and in the region around the Great Lakes known as the Corn Belt . The United States was a leader in seed improvement i.e. hybridization and in expanding uses for crops from the work of George Washington Carver to the development of bioplastics and biofuels . The mechanization of farming and intensive farming have been 

In [7]:
import csv

# for training data only

def get_type(annotation):
    if annotation['passage_answer']['candidate_index'] == -1:
        return 'na'
    if annotation['minimal_answer']['plaintext_start_byte'] > -1:
        return 'sa'
    if annotation['yes_no_answer'] != 'NONE':
        return 'bool'
    else:
        return 'la'
  
def verify_annotation_count(data, answer_type):
    
    counts = {'la':0,'boolean':0,'sa':0,'none':0, 'table':0, 'list':0, 'null':0}

    count = 0
    index = 0
    for example in data:
        index += 1
       
        annotation = example['annotations'][0]

        if annotation == None:
            continue

        if annotation['minimal_answer']['plaintext_start_byte'] != -1:
            continue
        counts[example['type'][0]] += 1
        if example['type'][0] != answer_type:
            continue
               
        passage_offsets = example['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
        passage_text = example['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
        
        num_sentences = len(passage_text.split("."))
        
        # num_sentences = 0
        # for sentence in sentences.sents:
        #     num_sentences += 1

        if num_sentences < 5:
            continue

        count += 1
        if count % 100 == 0:
            print(f"{count}/{index}")
    print(f"{count}/{index}")
    print(counts)

verify_annotation_count(train_data_long, 'la') 
verify_annotation_count(dev_data_long,'la')    

100/353
200/736
300/1116
400/1455
500/1793
600/2156
700/2519
800/2869
900/3197
1000/3551
1100/3887
1200/4225
1300/4585
1400/4947
1500/5299
1600/5656
1700/6042
1800/6400
1900/6728
2000/7088
2100/7450
2200/7759
2300/8113
2400/8516
2500/8854
2600/9223
2700/9578
2800/9931
2900/10294
3000/10647
3100/10962
3200/11306
3300/11616
3400/11997
3500/12334
3600/12636
3700/12981
3800/13348
3900/13729
4000/14082
4100/14382
4200/14750
4300/15084
4400/15448
4500/15823
4600/16143
4700/16513
4800/16840
4900/17216
5000/17619
5100/18047
5200/18403
5300/18785
5400/19159
5500/19553
5600/19881
5700/20209
5800/20563
5900/20889
6000/21230
6100/21580
6200/21908
6300/22277
6400/22648
6500/22989
6600/23361
6700/23697
6800/24072
6900/24442
7000/24805
7100/25130
7200/25487
7300/25834
7400/26195
7500/26588
7600/26947
7700/27301
7800/27693
7900/28066
8000/28386
8100/28732
8200/29083
8300/29399
8400/29753
8500/30118
8600/30428
8700/30790
8800/31208
8900/31606
9000/31894
9100/32244
9200/32587
9300/32988
9400/33339
9500/

In [30]:
print(len(train_data_full))
print(len(train_data_long))

307373
61499
